<a href="https://colab.research.google.com/github/alex-smith-uwec/CS-420/blob/main/SAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-sat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.9 MB/s eta 0:00:00


In [3]:
from sympy import symbols, Implies, Not, And, to_cnf
from pysat.solvers import Minisat22
import random

# Entering formulas with "natural" syntax"

In [4]:
 # Define the variables
a, b, c = symbols('a b c')

# Define the formula (¬a) -> (b ∧ c)
formula = Implies(Not(a), And(b, c))
##INaAbc in RPN

# Convert the formula to CNF using to_cnf function from sympy
cnf_formula = to_cnf(formula)

cnf_formula


(a | b) & (a | c)

In [5]:
print(cnf_formula)

(a | b) & (a | c)


# Converting from "natural language" representation of cnf to numerical format for solvers

In [6]:
# Initialize an empty list to store the clauses
cnf_list = []

# Create a mapping of variables to integers
var_map = {'a': 1, 'b': 2, 'c': 3}

# Traverse the CNF formula to populate the list of tuples
for clause in cnf_formula.args:
    # Initialize an empty list for the current clause
    clause_list = []
    for literal in clause.args:
        if literal.func is Not:
            # If the literal is a negation, use negative integers
            clause_list.append(-var_map[str(literal.args[0])])
        else:
            # Otherwise, use positive integers
            clause_list.append(var_map[str(literal)])
    # Convert the clause list to a tuple and append it to the CNF list
    cnf_list.append(clause_list)

cnf_list


[[1, 2], [1, 3]]

# Experiment witn Minisat22

In [6]:
# Create a solver instance
solver = Minisat22()

# Add clauses using add_clause method
# Positive integers represent the variable, and negative integers represent the negation of the variable
solver.add_clause([-1, 2])  # This represents the clause (¬A ∨ B)
solver.add_clause([1, -2])  # This represents the clause (A ∨ ¬B)
solver.add_clause([1, 2])   # This represents the clause (A ∨ B)

# Check for satisfiability
result = solver.solve()

# If the problem is satisfiable, you can get a model (a particular solution)
if result:
    model = solver.get_model()
    print("Satisfiable with model:", model)
else:
    print("Unsatisfiable")


Satisfiable with model: [1, 2]


In [7]:
# Number of symbols (variables)
n = 5
# Length of each clause
k = 3
# Number of clauses
m = 10

def generate_clause(n, k):
    """
    Generate a random clause of length k from n variables.
    """
    # Choose k different variables
    variables = random.sample(range(1, n+1), k)

    # Flip the sign of each variable with 50% probability
    clause = [var if random.choice([True, False]) else -var for var in variables]

    return clause

# Generate m clauses
clauses = [generate_clause(n, k) for _ in range(m)]
# print(clauses)
clauses

[[3, -5, 4],
 [1, 5, 3],
 [-1, 2, -4],
 [3, 2, -1],
 [-1, -2, -5],
 [3, -5, 1],
 [2, -4, -5],
 [-5, -3, 2],
 [-2, -5, 1],
 [-2, 5, 1]]

In [8]:
solver = Minisat22()

# Your clauses
# clauses = [[5, 10, 3], [10, 4, 6], [-9, -7, 4], [-4, -7, -1]]
# clauses=cnf_list

# Add clauses to the solver
for clause in clauses:
    solver.add_clause(clause)

# Check for satisfiability
result = solver.solve()

if result:
    model = solver.get_model()
    print("Satisfiable with model:", model)
else:
    print("Unsatisfiable")

Satisfiable with model: [1, 2, -3, -4, -5]


#N Queens

In [9]:
# Initialize SAT solver
solver = Minisat22()

N=8  # Size of the board

# Function to convert 2D board coordinates to 1D variable names
def var(i, j):
    return i * N + j + 1  # +1 because SAT variables start from 1

# Generate row constraints using list comprehension
row_clauses = [[-var(i, j), -var(i, k)] for i in range(N) for j in range(N) for k in range(N) if j != k]
##list comprehension above is equivalent to this loop
# Generate constraints for each row
# for i in range(N):
#     for j in range(N):
#         for k in range(N):
#             if j != k:
#                 solver.add_clause([-var(i, j), -var(i, k)])

# Generate column constraints using list comprehension
col_clauses = [[-var(i, j), -var(k, j)] for j in range(N) for i in range(N) for k in range(N) if i != k]

# Generate diagonal constraints using list comprehension
diag_clauses = [[-var(i, j), -var(k, l)] for i in range(N) for j in range(N)
                for k in range(N) for l in range(N)
                if i != k and j != l and abs(i - k) == abs(j - l)]

# Generate at least one queen per column clauses
one_queen_per_col_clauses = [[var(i, j) for i in range(N)] for j in range(N)]

# Combine all sets of clauses
all_clauses = row_clauses + col_clauses + diag_clauses + one_queen_per_col_clauses

# Add clauses to the solver
for clause in all_clauses:
    solver.add_clause(clause)

# Solve and print the solution
if solver.solve():
    model = solver.get_model()
    print("Satisfiable with model:", model)
else:
    print("Unsatisfiable")


Satisfiable with model: [-1, -2, -3, -4, -5, 6, -7, -8, -9, -10, -11, 12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, 23, -24, 25, -26, -27, -28, -29, -30, -31, -32, -33, -34, 35, -36, -37, -38, -39, -40, -41, -42, -43, -44, 45, -46, -47, -48, -49, 50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, 64]


In [10]:
clean=[k for k in model if k>0]

In [14]:
clean

[6, 12, 23, 25, 35, 45, 50, 64]

In [18]:
len(all_clauses)

1464

In [27]:
unique_clauses = set(tuple(clause) for clause in all_clauses)
len(unique_clauses)


1464